In [1]:
import networkx as nx
import Levenshtein as LD
import numpy as np
import pandas as pd
#import pygraphviz
import matplotlib.pyplot as plt
from pyvis.network import Network
from scipy.stats import *
import seaborn as sns

In [4]:
def generate_dist_matrix(d):
    num_CDR3 = len(d)
    dist_mt = np.zeros((num_CDR3,num_CDR3))
    for i, cdr3_i in enumerate(d):
        for j,cdr3_j in enumerate(d):
            if i!=j:
                distance = LD.distance(cdr3_i,cdr3_j)
                max_len = max(len(cdr3_i),len(cdr3_j))
                r = distance/max_len
                if r>0.3:
                    continue
                dist_mt[i,j] = r
                dist_mt[j,i] = r
    return pd.DataFrame(dist_mt,columns = d,index=d)
def binding(x):
    n = x.count('+')
    if n==0:
        return -6
    if n>4:
        n=4
    return n-4

In [3]:
data = pd.read_excel("Pan.xlsx")

In [16]:
otherNb = pd.read_excel("OtherNb.xlsx")

In [17]:
otherNb['SARS2 binding'] = otherNb['SARS-CoV-2'].apply(lambda x:binding(x))
otherNb['SARS1 binding']= otherNb['SARS-CoV'].apply(lambda x:binding(x))
otherNb['RmYN02 binding']= otherNb['RmYN02-CoV'].apply(lambda x:binding(x))
otherNb['BM4831 binding']= otherNb['BM-4831-CoV'].apply(lambda x:binding(x))
otherNb['binding score'] = (otherNb[[i for i in otherNb.columns if ' binding' in i]].mean(1)+6)/6

In [18]:
otherNb.to_csv("otherNb_binding_score.csv")

In [5]:
data['SARS2 binding'] = data['SARS2'].apply(lambda x:binding(x))
data['SARS1 binding']= data['SARS1'].apply(lambda x:binding(x))
data['RmYN02 binding']= data['RmYN02'].apply(lambda x:binding(x))
data['BM4831 binding']= data['BM4831'].apply(lambda x:binding(x))
data['binding score'] = (data[[i for i in data.columns if ' binding' in i]].mean(1)+6)/6

In [6]:
data['log(Neut)'] = data['Neut(nM)'].apply(lambda x: np.log10(x))

In [7]:
#bmin = data['binding score'].min()
#bmax = data['binding score'].max()
data['Pan_1'] = data['binding score'].apply(lambda x: x)
data['Pan_2'] = data['binding score'].apply(lambda x: np.exp(-x))

In [8]:
data.to_csv("psNb_binding_score.csv")

In [70]:
data = data[data['#']!='5p-77']
data = data[data['#']!='4P-16']

In [16]:
data = data.sort_values(['CDR','binding score'],ascending=False)

In [38]:
data = data.drop_duplicates("CDR")

In [49]:
all_CDR = data['CDR'].values
names = data['#'].values
CDR_Name_dict = dict(data[['CDR','#']].values)
CDR_Name2_dict = dict(data[['CDR','New #']].values)
CDR_Pan1_dict = dict(data[['CDR','Pan_1']].values)
CDR_Pan2_dict = dict(data[['CDR','Pan_2']].values)
CDR_SEC_dict = dict(data[['CDR','SEC epitope group']].values)


CDR_Neut_dict = dict(data[['CDR','log(Neut)']].values)
CDR_binding_dict = dict(data[['CDR','binding score']].values)

In [33]:
len(all_CDR)

100

In [19]:
# data['CDR1'] = data['Original Amino Acid Sequence'].apply(lambda x: find_cdr1(x))
# data['CDR2'] = data['Original Amino Acid Sequence'].apply(lambda x: find_cdr2(x))
# data['CDR3'] = data['Original Amino Acid Sequence'].apply(lambda x: find_cdr3(x))

In [50]:
dist_matrix = generate_dist_matrix(all_CDR)
graph_all = nx.from_numpy_matrix(dist_matrix.values)
mapping = dict(zip(graph_all,all_CDR))
graph_all = nx.relabel_nodes(graph_all,mapping)

In [51]:
G = nx.Graph()
for n in graph_all:
    crossreactivity1 = CDR_Pan1_dict[n]
    crossreactivity2 = CDR_Pan2_dict[n]
    sec = CDR_SEC_dict[n]
    name = CDR_Name_dict[n]
    neuts = int(np.floor(CDR_Neut_dict[n]))
    name2 = CDR_Name2_dict[n]
    G.add_node(n, sequence=n,name=name,name2=name2,pan1=crossreactivity1,pan2=crossreactivity2,Neut=neuts,sec=sec)
for e in graph_all.edges:
    G.add_edge(e[0],e[1])

In [52]:
nx.write_gml(G, 'PsNb_1220.gml')

In [32]:
data['Pan_2'].min()

5.754602676005731

In [99]:
data1 = data.set_index("#")

In [116]:
len("GGWGLTQPISVDY")

13

In [117]:
len("GGNHYDPSKYYIADEYDY")

18